# ARSLM — Notebook (cleaned)

Ce notebook a été nettoyé pour utiliser le module `arslm_fixes.py` présent à la racine du dépôt.

Important : redémarrez le kernel avant d'exécuter les cellules (Runtime -> Restart runtime).


In [ ]:
# Installer la dépendance minimale (si nécessaire)
!pip install -q transformers


In [ ]:
# Importer les classes corrigées depuis arslm_fixes.py (fichier ajouté dans le dépôt)
from arslm_fixes import AdvancedTokenizer, ARSCell, ARSLM, collate_batch, train_demo
import torch

print('torch version:', torch.__version__)

# Smoke test: tokenizer + forward pass avec un petit contexte
tokenizer = AdvancedTokenizer()
print('vocab size =', len(tokenizer))

# Préparer un batch simple
context = 'hello world'
input_ids = torch.tensor([[tokenizer.bos_token_id] + tokenizer.encode(context)], dtype=torch.long)
model = ARSLM(tokenizer, emb_dim=64, hidden_dim=128, num_layers=2)
model.eval()
with torch.no_grad():
    logits, gates = model(input_ids)
    print('logits shape =', logits.shape)
    print('gates shape =', gates.shape)


In [ ]:
# Courte démonstration d'entraînement (n_epochs=2 pour un test rapide)
model, tokenizer = train_demo(n_epochs=2)

# Génération d'exemple
idx = torch.tensor([[tokenizer.bos_token_id] + tokenizer.encode('hello world')], dtype=torch.long)
out = model.generate(idx, max_new_tokens=10, temperature=1.0)[0]
print('Generated:', tokenizer.decode(out))


## Remarques
- Le module `arslm_fixes.py` contient les définitions corrigées (AdvancedTokenizer, ARSCell, ARSLM, collate_batch, train_demo).
- Pour une utilisation prolongée, augmentez `n_epochs` dans `train_demo` ou utilisez votre propre routine d'entraînement.
- Si vous exécutez sur Colab/GPU, assurez-vous d'avoir activé l'accélérateur GPU dans Runtime -> Change runtime type.
